# Feature data processing

This notebook is to generate a feature table (where peaks are the rows and each column shows if a transcription factor is found at that peak)

- We will break it into steps

These are our important files 

- The peak ids from our experiment and thier locations are in
`/project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/all_basson_cerebellarWTminR123_peaks.bed`
- The overlap of these peaks with the ChIPAtlas dataset (Our 'long' datatable) is in 
`/project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/peak_call_data.dir/basson_overlap_with_chipatlas_TF.bed`
    - This contains our peak ids (e.g. atac_de_peak_chr1_3094800_3095300) and what datasets they overlap with (SRX5237754) **NOTE THIS FILE IS TOO BIG TO DO MUCH WITH USE OTHER FILES INSTEAD**
    
- This file describes the peak datasets for each tissue/factor and gives the file name of how to download them
    - `/project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/peak_call_data.dir/fileList.tab`
    - We will use this file instead to get the names of the bedfiles to download and overlap and details about whats in the files


In [4]:
import pandas as pd

Read in the files that summarise the data we want to overlap with our peaks 

In [5]:
peak_files = pd.read_table('/project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/peak_call_data.dir/fileList.tab',header=None)
peak_files.columns = ['file_name','genome','data_type','antigen','cell_type_class','cell_type','threshold','experimental_ids'] 

In [6]:
# subset the peakfiles to the mouse (mm10) genome 
peak_files_mm10 = peak_files[peak_files['genome']=='mm10'].copy()
# look at datatypes - we only want TF and others --> 143000 cell types 
peak_files_mm10.groupby('data_type').count().reset_index()

,data_type,file_name,genome,antigen,cell_type_class,cell_type,threshold,experimental_ids
0,ATAC-Seq,2520,2520,2520,2520,2516,2520,2451
1,Bisulfite-Seq,308,308,308,308,307,308,307
2,DNase-seq,576,576,576,576,572,576,551
3,Histone,5040,5040,5040,5040,5036,5040,5028
4,Input control,2548,2548,2548,2548,2548,2548,2532
5,No description,860,860,768,860,856,860,844
6,RNA polymerase,928,928,928,928,928,928,924
7,TFs and others,14300,14300,14300,14300,14300,14300,14268
8,Unclassified,1596,1596,1596,1596,1596,1596,1592


In [7]:
# subset this table to TF (transcription factors) and see what tissues there are in the dataset 
peak_files_mm10_TFs = peak_files_mm10[peak_files_mm10['data_type']=='TFs and others'].copy()

In [10]:
pd.set_option('display.max_rows', 500)

In [12]:
peak_files_mm10_TFs.groupby(['cell_type_class','cell_type']).count()

file_name  genome  data_type  antigen   
cell_type_class cell_type                                                      
Adipocyte       -                            164     164        164      164  \
                Adipocytes                     4       4          4        4   
                Adipose Tissue                 4       4          4        4   
                Brown adipocytes               4       4          4        4   
                Brown preadipocytes            4       4          4        4   
...                                          ...     ...        ...      ...   
Uterus          -                             52      52         52       52   
                Decidua                        4       4          4        4   
                Endometriotic lesions          4       4          4        4   
                Endometrium                    4       4          4        4   
                Uterus                         4       4          4        4   

                                       threshold  experimental_ids  
cell_type_class cell_type                                           
Adipocyte       -                            164               160  
                Adipocytes                     4                 4  
                Adipose Tissue                 4                 4  
                Brown adipocytes               4                 4  
                Brown preadipocytes            4                 4  
...                                          ...               ...  
Uterus          -                             52                52  
                Decidua                        4                 4  
                Endometriotic lesions          4                 4  
                Endometrium                    4                 4  
                Uterus                         4                 4  

[549 rows x 6 columns]

What are the cell types we want out of this - all the cerebellum ones and granule neuron progenitors and neural progenitors? Or all of them? Lets go for all of them for now we can always subset to just the neuronal ones later

How many different antigens (TFs) are there 

In [13]:
print(peak_files_mm10_TFs.antigen.nunique())


884


In [14]:
print(peak_files_mm10_TFs.antigen.unique())


['-' 'GFP' 'Etv5' 'Bnc1' 'Foxo1' 'Smarca4' 'Esrrb' 'Rreb1' 'Rb1' 'Cebpb'
 'T' 'Hnf4a' 'Hdac4' 'Yy2' 'Kdm5c' 'Sp6' 'Smarcc1' 'Mbd3' 'Ehmt2' 'Gata4'
 'Hnf1b' 'Tbx3' 'Thrap3' 'Ctr9' 'Supt5' 'Rpa2' 'Hnf1a' 'Wt1' 'Ar' 'Smad3'
 'Smc5' 'Kdm4d' 'Nfatc1' 'Satb2' 'Tcf7l2' 'Hcfc1' 'Top2b' 'Bmi1' 'E2f3'
 'Nfkbia' 'Ncor1' 'Hsf1' 'Arntl' 'Ezh2' 'Smad4' 'Gtf2f1' 'Ing5' 'Kdm6b'
 'Elf4' 'Rara' 'Gabpa' 'Uhrf2' 'Irf7' 'Ddx21' 'Hif1a' 'Sp9' 'Nfkb1'
 'Biotin' 'Trim28' 'Bcl6' 'Foxa2' 'Tcf3' 'Cdx2' 'Cbx7' 'Ddx5' 'Rest'
 'Ctf1' 'E4f1' 'Taf7l' 'Tcf7' 'Pou3f2' 'Wdr82' 'Smc1a' 'Crebbp' 'Duxbl1'
 'Ago2' 'Hoxc8' 'Mecp2' 'Nfe2l2' 'Sin3b' 'Rnf2' 'Esrra' 'Mef2c' 'Ncaph2'
 'Phf6' 'Nfia' 'Junb' 'Onecut2' 'Alkbh5' 'Nelfa' 'Klf4' 'Epitope tags'
 'Wdr61' 'Smc3' 'Ikzf1' 'Myc' 'Suz12' 'Foxp1' 'Pcgf1' 'Ctnnb1' 'L3mbtl2'
 'Nbn' 'Zfp191' 'Sox17' 'Sox4' 'Irf4' 'Ncoa3' 'Sox2' 'Zbtb2' 'Brd4' 'Ctcf'
 'Sox9' 'Arid1b' 'Kat2a' 'Zeb1' 'Srf' 'Matr3' 'Ebf2' 'Trp53' 'Irf9'
 'Prox1' 'Med1' 'Ssu72' 'Brd3' 'Sp1' 'Safb' 'Usf2' 'Hnf4g' 'Ets1'

We want to download the peaks for each of these factors - we need to get the file names from the table to see what they are specified as

In [15]:
# have a look at what the neural file names are like 
peak_files_mm10_TFs[peak_files_mm10_TFs['cell_type_class']=='Neural'].head(100)

,file_name,genome,data_type,antigen,cell_type_class,cell_type,threshold,experimental_ids
129384,Oth.Neu.20.AllAg.AllCell,mm10,TFs and others,-,Neural,-,20,"SRX661585,SRX5287318,SRX19085827,SRX19085833,S..."
129656,Oth.Neu.10.AllAg.AllCell,mm10,TFs and others,-,Neural,-,10,"SRX2463202,SRX661585,SRX2957352,SRX2424693,SRX..."
129864,Oth.Neu.05.AllAg.AllCell,mm10,TFs and others,-,Neural,-,05,"SRX2463202,SRX14039259,SRX14039261,SRX10394555..."
129898,Oth.Neu.50.AllAg.AllCell,mm10,TFs and others,-,Neural,-,50,"SRX661585,SRX5287318,SRX19085827,SRX19085833,S..."
130289,Oth.Neu.10.Hif1a.AllCell,mm10,TFs and others,Hif1a,Neural,-,10,"SRX3331740,SRX3331743,SRX3331741,SRX3331742"
130351,Oth.Neu.10.Pou3f2.AllCell,mm10,TFs and others,Pou3f2,Neural,-,10,SRX323573
130381,Oth.Neu.05.Esrra.AllCell,mm10,TFs and others,Esrra,Neural,-,05,"SRX1743136,SRX1743135"
130382,Oth.Neu.50.Mef2c.AllCell,mm10,TFs and others,Mef2c,Neural,-,50,"SRX10616112,SRX10616116,SRX10616117,SRX10616113"
130385,Oth.Neu.10.Nfia.AllCell,mm10,TFs and others,Nfia,Neural,-,10,"SRX7895559,SRX7895558,SRX11929789,SRX11929790,..."
130404,Oth.Neu.20.AllAg.Olfactory_Nerve,mm10,TFs and others,-,Neural,Olfactory Nerve,20,"SRX5287318,SRX3828419,SRX3828418,SRX3828415,SR..."


In [16]:
#subset again to get all the peaks with a threshold of 05 - this is more relaxed then the other thresholds
peak_files_mm10_TFs_05 = peak_files_mm10_TFs[peak_files_mm10_TFs['file_name'].str.contains('05')]
print(len(peak_files_mm10_TFs_05))

3578


In [18]:
# get a list of the all filenames for the factors and all their peaks for all tissues 
files_for_download = peak_files_mm10_TFs_05[(peak_files_mm10_TFs_05['cell_type_class']=='All cell types') & (peak_files_mm10_TFs_05['antigen'] != '-')]['file_name'].unique()



In [20]:
print(files_for_download)

['Oth.ALL.05.Foxo1.AllCell' 'Oth.ALL.05.Gtf2f1.AllCell'
 'Oth.ALL.05.Ddx21.AllCell' 'Oth.ALL.05.Sp9.AllCell'
 'Oth.ALL.05.Nfkb1.AllCell' 'Oth.ALL.05.E4f1.AllCell'
 'Oth.ALL.05.Taf7l.AllCell' 'Oth.ALL.05.Foxa2.AllCell'
 'Oth.ALL.05.Pcgf1.AllCell' 'Oth.ALL.05.Nbn.AllCell'
 'Oth.ALL.05.Arid1b.AllCell' 'Oth.ALL.05.Matr3.AllCell'
 'Oth.ALL.05.Bcl11b.AllCell' 'Oth.ALL.05.Tnni2.AllCell'
 'Oth.ALL.05.Ctbp2.AllCell' 'Oth.ALL.05.Taf9b.AllCell'
 'Oth.ALL.05.Srsf1.AllCell' 'Oth.ALL.05.Creb3l2.AllCell'
 'Oth.ALL.05.0610010K14Rik.AllCell' 'Oth.ALL.05.Gcm1.AllCell'
 'Oth.ALL.05.Cbx4.AllCell' 'Oth.ALL.05.Wtap.AllCell'
 'Oth.ALL.05.Mybl1.AllCell' 'Oth.ALL.05.Prdm15.AllCell'
 'Oth.ALL.05.Cdx2.AllCell' 'Oth.ALL.05.Hdac2.AllCell'
 'Oth.ALL.05.Cbfa2t3.AllCell' 'Oth.ALL.05.Sox2.AllCell'
 'Oth.ALL.05.Kmt2c.AllCell' 'Oth.ALL.05.Ebf4.AllCell'
 'Oth.ALL.05.DNA-RNA_hybrid.AllCell' 'Oth.ALL.05.Lin9.AllCell'
 'Oth.ALL.05.AML1-ETO.AllCell' 'Oth.ALL.05.Dmap1.AllCell'
 'Oth.ALL.05.Tbx3.AllCell' 'Oth.ALL.05.Irf3.AllCe

Then I ran this code to download the files using os.system to run a linux command that downloads the datasets using wget and then compresses it using gzip

I've turned this command to markdown becuase we don't want to run it again and redownload everything :) 


```
# This is python code to download the files 
path_to_files = '/project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/peak_call_data.dir/indiv_factors_all_cells.dir/'

import os
for file in files_for_download:
    os.system(f'wget -P {path_to_files} https://chip-atlas.dbcls.jp/data/mm10/assembled/{file}.bed && gzip {path_to_files}{file}.bed')
    
```

I then ran some linux command line commands and 
1. used the bedtools program [here](https://bedtools.readthedocs.io/en/latest/content/tools/intersect.html) to merge all the peaks from the different tissues together if they overlapped (this makes the file smaller) using `bedtools merge`. 
2. I then compressed these using `gzip`
3. Then used `bedtools intersect` to overlap all the atac peaks from our study with the peaks of the different transcription factors, and output a file of those peaks that were found to overlap those factors
4. Finally then compressed these files using `gzip` - these files are now all in `/project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/peak_call_data.dir/indiv_factors_all_cells.dir/intersection_with_basson_peaks.dir`
5. Then i tidyied up the filenames using `mv` 

Here is the linux code: 

```
cd /project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/peak_call_data.dir/indiv_factors_all_cells.dir

for file in *.bed.gz; do bedtools merge $file > $file.merged.bed; done 

gzip *.merged.bed

for f in *merged.bed.gz; do bedtools intersect -a ../../../all_basson_cerebellarWTminR123_peaks.bed -b $f  -wa > intersection_with_basson_peaks.dir/basson_cere_WTmin123_in_$f.bed; done

gzip *bed

for f in ./*; do mv "$f" "${f%.bed.gz.merged.bed.gz.bed.gz}.bed.gz" ; done
```


You can see all the origional downloaded files and thier merged counterparks here:

In [21]:
! ls /project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/peak_call_data.dir/indiv_factors_all_cells.dir

intersection_with_basson_peaks.dir
Oth.ALL.05.0610010K14Rik.AllCell.bed.gz
Oth.ALL.05.0610010K14Rik.AllCell.bed.gz.merged.bed.gz
Oth.ALL.05.5-hmC.AllCell.bed.gz
Oth.ALL.05.5-hmC.AllCell.bed.gz.merged.bed.gz
Oth.ALL.05.5-mC.AllCell.bed.gz
Oth.ALL.05.5-mC.AllCell.bed.gz.merged.bed.gz
Oth.ALL.05.8-Hydroxydeoxyguanosine.AllCell.bed.gz
Oth.ALL.05.8-Hydroxydeoxyguanosine.AllCell.bed.gz.merged.bed.gz
Oth.ALL.05.Acaa2.AllCell.bed.gz
Oth.ALL.05.Acaa2.AllCell.bed.gz.merged.bed.gz
Oth.ALL.05.Acss2.AllCell.bed.gz
Oth.ALL.05.Acss2.AllCell.bed.gz.merged.bed.gz
Oth.ALL.05.Actb.AllCell.bed.gz
Oth.ALL.05.Actb.AllCell.bed.gz.merged.bed.gz
Oth.ALL.05.Adnp.AllCell.bed.gz
Oth.ALL.05.Adnp.AllCell.bed.gz.merged.bed.gz
Oth.ALL.05.ADP-ribose.AllCell.bed.gz
Oth.ALL.05.ADP-ribose.AllCell.bed.gz.merged.bed.gz
Oth.ALL.05.Aebp2.AllCell.bed.gz
Oth.ALL.05.Aebp2.AllCell.bed.gz.merged.bed.gz
Oth.ALL.05.Aff3.AllCell.bed.gz
Oth.ALL.05.Aff3.AllCell.bed.gz.merged.bed.gz
Oth.ALL.05.Aff4.AllCell.bed.gz
Oth.ALL.05.Aff4.AllCel

Each of these files we downloaded contains data from several different experiments - the full details of them are in the expt_data table below - we might need to use this at some point

In [ ]:
# I've commented this out as otherwise the notebook was to big to share on git, but feel free to uncomment and look at it
#exp_data = pd.read_csv('/project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/experimentList_SRXonly.tab',names=range(50),sep='\t')

In [ ]:
# subset it to the mouse genome (mm10)
#exp_data_mm10_TFs = exp_data_mm10[exp_data_mm10[2] =='TFs and others']
#exp_data_mm10_TFs.head()

And here are all our peaks that overlap the files we downloaded - these are the files you will be working with to make the features table  

In [22]:
! ls /project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/peak_call_data.dir/indiv_factors_all_cells.dir/intersection_with_basson_peaks.dir

basson_cere_WTmin123_in_Oth.ALL.05.0610010K14Rik.AllCell.bed.gz
basson_cere_WTmin123_in_Oth.ALL.05.5-hmC.AllCell.bed.gz
basson_cere_WTmin123_in_Oth.ALL.05.5-mC.AllCell.bed.gz
basson_cere_WTmin123_in_Oth.ALL.05.8-Hydroxydeoxyguanosine.AllCell.bed.gz
basson_cere_WTmin123_in_Oth.ALL.05.Acaa2.AllCell.bed.gz
basson_cere_WTmin123_in_Oth.ALL.05.Acss2.AllCell.bed.gz
basson_cere_WTmin123_in_Oth.ALL.05.Actb.AllCell.bed.gz
basson_cere_WTmin123_in_Oth.ALL.05.Adnp.AllCell.bed.gz
basson_cere_WTmin123_in_Oth.ALL.05.ADP-ribose.AllCell.bed.gz
basson_cere_WTmin123_in_Oth.ALL.05.Aebp2.AllCell.bed.gz
basson_cere_WTmin123_in_Oth.ALL.05.Aff3.AllCell.bed.gz
basson_cere_WTmin123_in_Oth.ALL.05.Aff4.AllCell.bed.gz
basson_cere_WTmin123_in_Oth.ALL.05.Ago2.AllCell.bed.gz
basson_cere_WTmin123_in_Oth.ALL.05.Ahcy.AllCell.bed.gz
basson_cere_WTmin123_in_Oth.ALL.05.Ahr.AllCell.bed.gz
basson_cere_WTmin123_in_Oth.ALL.05.Aicda.AllCell.bed.gz
basson_cere_WTmin123_in_Oth.ALL.05.Aire.AllCell.bed.gz
basson_cere_WTmin123_in_Oth

If you have a look at the top of these files you will see there are different columns 

In [23]:
overlap_of_our_peaks_with_Zscan5b_AllCell = pd.read_table('/project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/peak_call_data.dir/indiv_factors_all_cells.dir/intersection_with_basson_peaks.dir/basson_cere_WTmin123_in_Oth.ALL.05.Zscan5b.AllCell.bed.gz',header=None)

In [24]:
overlap_of_our_peaks_with_Zscan5b_AllCell.head()

,0,1,2,3
0,chr1,171038166,171038666,atac_de_peak_chr1_171038166_171038666
1,chr11,3126968,3127468,atac_de_peak_chr11_3126968_3127468
2,chr11,3153586,3154086,atac_de_peak_chr11_3153586_3154086
3,chr11,48834196,48834696,atac_de_peak_chr11_48834196_48834696
4,chr12,78350732,78351232,atac_de_peak_chr12_78350732_78351232


In [ ]:
# assign column names 
overlap_of_our_peaks_with_Zscan5b_AllCell.columns = ['chr','start','end','peak_id']
overlap_of_our_peaks_with_Zscan5b_AllCell.head()

TODO: 
    
1. In a new notebook (that you call `2_making_feature_table` use pd.read_csv() to read in our full list of peaks (>112,000) - use len or .shape to check the length so its the right file 
2. Get a list of all the files in `/project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/peak_call_data.dir/indiv_factors_all_cells.dir/intersection_with_basson_peaks.dir`

3. For one file at first work out how to extract the peak_id column from the overlaps file and make a new column on the dataframe containing all our peaks to indictcate using 1 or 0 if that peak overlaps with that TF  (the peak_ids are the same format between the two files) - make sure the column is labeled with the TF name (e.g. Zscan5b)

4. Once that works for one file, loop over all the files in your list of all the overlaps to add a column for every TF 
5. Write your final table out to the  `/project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/peak_call_data.dir/feature_table.dir` directory - when you write it our use compression='gzip' to make it smaller and take up less space. 

Good luck :) and feel free to ask me any questions if your stuck :) 
